# Greenland peripherial glacier pre-image download processing

#### Jukes Liu
__Last modified 10-15-2019.__

## 1) Import packages, set base path, set glaciers of interest by BoxID

In [103]:
import subprocess
import numpy as np
import os
import pandas as pd
import rasterio
import fiona
from shapely.geometry import Polygon, Point
import shapely
import math
import shutil
from PIL import Image
import matplotlib.image as mpimg
import scipy.misc

#SET basepath to your own folder
basepath='/home/jukes/Documents/Sample_glaciers/'
downloadpath = '/media/jukes/jukes1/LS8aws/'

#ENTER list of glaciers of interest by BoxID
#make this into a widget where you can enter them in?
BOXIDS = ['001', '002', '004', '033', '120', '174', '235', '259', '277', '531'];
# BOXIDS = ['Alison', 'Helheim']
# BOXIDS = ['147', '148', '149', '150', '152', '190', '191', '192', '193', '194', '195', '195', '196', '213', '214', '215']

In [3]:
def distance(x1, y1, x2, y2):
    dist = math.sqrt(((x2-x1)**2) + ((y2-y1)**2))
    return dist

In [108]:
#define a function that uses trigonometry to calculate the slope angle
#that each pair of box vertices make
def slope_angle(x1, y1, x2, y2):
    return np.arctan2(y2-y1, x2-x1)*180/np.pi

## 2) Create buffer zone around terminus boxes and rasterize/subset terminus boxes

The following code pulls the buffer distances around the terminus boxes from an existing .csv file with the exported attributes tables for the peripheral glacier terminus boxes. These buffer distances will be used to create a buffer zone to subset the Landsat scenes.

In [5]:
buffers = []

#Calculate a buffer distance around the terminus box using the UTM projected boxes
for BoxID in BOXIDS:
    buff_distances = []

    for file in os.listdir(basepath+'Box'+BoxID+'/'):
        if 'UTM' in file and '.shp' in file:
            print(file)
            boxpath = basepath+"Box"+BoxID+"/"+file
#             print(boxpath)
            
            termbox = fiona.open(boxpath)
            #grab the box feature:
            box = termbox.next()
            box_geom= box.get('geometry')
            box_coords = box_geom.get('coordinates')[0]
#             print(box_geom)
            
            points = []
            for coord_pair in box_coords:
                lat = coord_pair[0]
                lon = coord_pair[1]
                
                points.append([lat, lon])
            
            #Calculate distance between 1 and 2 and distance between 2 and 3
            #pick the longer one (length)
            coord1 = points[0]
            coord2 = points[1]
            coord3 = points[2]
            
            #1 and 2
            dist1 = distance(coord1[0], coord1[1], coord2[0], coord2[1])       
            #2 and 3
            dist2 = distance(coord2[0], coord2[1], coord3[0], coord3[1])
            
            buff_dist = int(np.max([dist1, dist2]))
#             print(buff_dist)
            buff_distances.append(buff_dist)
    
    buffer = buff_distances[0]
    buffers.append(buffer)

buff_df = pd.DataFrame(list(zip(BOXIDS, buffers)), columns=['BoxID', 'Buff_dist_m'])
buff_df

Box147_UTM_23.shp
Box147_UTM_24.shp
Buffer147_UTM_23.shp
Buffer147_UTM_24.shp
Box148_UTM_23.shp
Box148_UTM_24.shp
Buffer148_UTM_24.shp
Buffer148_UTM_23.shp
Box149_UTM_24.shp
Box149_UTM_23.shp
Buffer149_UTM_23.shp
Buffer149_UTM_24.shp
Buffer150_UTM_24.shp
Box150_UTM_24.shp
Box150_UTM_23.shp
Buffer150_UTM_23.shp
Box152_UTM_23.shp
Buffer152_UTM_24.shp
Box152_UTM_24.shp
Buffer152_UTM_23.shp
Box190_UTM_24.shp
Buffer190_UTM_24.shp
Box190_UTM_23.shp
Buffer190_UTM_23.shp
Box191_UTM_24.shp
Box191_UTM_23.shp
Buffer191_UTM_23.shp
Buffer191_UTM_24.shp
Buffer192_UTM_23.shp
Buffer192_UTM_24.shp
Box192_UTM_23.shp
Box192_UTM_24.shp
Box193_UTM_23.shp
Buffer193_UTM_23.shp
Buffer193_UTM_24.shp
Box193_UTM_24.shp
Buffer194_UTM_24.shp
Box194_UTM_23.shp
Box194_UTM_24.shp
Buffer194_UTM_23.shp
Buffer195_UTM_23.shp
Buffer195_UTM_24.shp
Box195_UTM_24.shp
Box195_UTM_23.shp
Buffer195_UTM_23.shp
Buffer195_UTM_24.shp
Box195_UTM_24.shp
Box195_UTM_23.shp
Buffer196_UTM_24.shp
Buffer196_UTM_23.shp
Box196_UTM_23.shp
Box1

/home/jukes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  from ipykernel import kernelapp as app


,BoxID,Buff_dist_m
0,147,1083
1,148,861
2,149,662
3,150,34
4,152,934
5,190,1946
6,191,1685
7,192,33
8,193,1509
9,194,87


The next section creates a buffer zone using GDAL command **ogr2ogr** with the following syntax:

    ogr2ogr Buffer###.shp path_to_terminusbox###.shp  -dialect sqlite -sql "SELECT ST_Buffer(geometry, buffer_distance) AS geometry,*FROM 'Box###'" -f "ESRI Shapefile"

In [6]:
for index, row in buff_df.iterrows():
    BoxID = row['BoxID']
    buff_dist = str(row['Buff_dist_m'])
    
    #SET path to the terminus box shapefiles
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp"
    outputbuffer_path = basepath+"Box"+BoxID+"/Buffer"+BoxID+".shp"
    
    #SET buffer command and print to check it
    buffer_cmd = 'ogr2ogr '+outputbuffer_path+" "+terminusbox_path+' -dialect sqlite -sql "SELECT ST_Buffer(geometry, '+buff_dist+") AS geometry,*FROM 'Box"+BoxID+"'"+'" -f "ESRI Shapefile"'
    print(buffer_cmd)
    
    subprocess.call(buffer_cmd, shell=True)
    
    print("Box"+BoxID)

ogr2ogr /home/jukes/Documents/Sample_glaciers/Box147/Buffer147.shp /home/jukes/Documents/Sample_glaciers/Box147/Box147.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 1083) AS geometry,*FROM 'Box147'" -f "ESRI Shapefile"
Box147
ogr2ogr /home/jukes/Documents/Sample_glaciers/Box148/Buffer148.shp /home/jukes/Documents/Sample_glaciers/Box148/Box148.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 861) AS geometry,*FROM 'Box148'" -f "ESRI Shapefile"
Box148
ogr2ogr /home/jukes/Documents/Sample_glaciers/Box149/Buffer149.shp /home/jukes/Documents/Sample_glaciers/Box149/Box149.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 662) AS geometry,*FROM 'Box149'" -f "ESRI Shapefile"
Box149
ogr2ogr /home/jukes/Documents/Sample_glaciers/Box150/Buffer150.shp /home/jukes/Documents/Sample_glaciers/Box150/Box150.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 34) AS geometry,*FROM 'Box150'" -f "ESRI Shapefile"
Box150
ogr2ogr /home/jukes/Documents/Sample_glaciers/Box152/Buffer152.shp /home

The terminus box shapefiles are then rasterized (to be used as a mask during the WTMM filering) using the GDAL **gdal_rasterize** command and subset to the buffer zone using the GDAL **gdalwarp** command using the following syntax:

1) Rasterize

    gdal_rasterize -burn 1.0 -tr x_resolution y_resolution -a_nodata 0.0 path_to_terminusbox.shp path_to_terminusbox_raster.TIF

The x_resolution and y_resolution are set to be 15.0 (meters) to match the Landsat B8 resolution.
    
2) Subset

    gdalwarp -cutline path_to_Buffer###.shp -crop_to_cutline path_to_terminusbox_raster.TIF path_to_subset_raster_cut.TIF

In [27]:
for index, row in buff_df.iterrows():
    BoxID = row['BoxID']
    #SET path to the terminus box shapefiles
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp"
    buffer_path = basepath+"Box"+BoxID+"/Buffer"+BoxID+".shp"
    
    #output raster path:
    terminusraster_path = basepath+"Box"+BoxID+"/Box"+BoxID+".TIF"
    cutraster_path = basepath+"Box"+BoxID+"/Box"+BoxID+"_cut.TIF"
    
    #SET commands and print to check
    rasterize_cmd = 'gdal_rasterize -burn 1.0 -tr 15.0 15.0 -a_nodata 0.0 '+terminusbox_path+' '+terminusraster_path
    subsetbuffer_cmd = 'gdalwarp -cutline '+buffer_path+' -crop_to_cutline '+terminusraster_path+" "+cutraster_path
    #print(export_GDALpath+rasterize_cmd)
    #print(export_GDALpath+subsetbuffer_cmd)
    
    #RASTERIZE & SUBSET
    subprocess.call(rasterize_cmd, shell=True)
    subprocess.call(subsetbuffer_cmd, shell=True)
    
    print("Box"+BoxID)
    

Box147
Box148
Box149
Box150
Box152
Box190
Box191
Box192
Box193
Box194
Box195
Box195
Box196
Box213
Box214
Box215


## 3) Calculate average flow direction (weighted by magnitude) for each glacier

The following code processes 2016-2017 ice velocity data from the ESA Cryoportal to determine each glacier of interest's weighted average flow direction. The ice velocity direction (calculated from yx velocity) and the velocity magnitude at each glacier's terminus  is subset using the terminus box shapefile using a GDAL command (**gdalwarp**) with the following syntax:

    gdalwarp -cutline path_to_terminusbox.shp -crop_to_cutline path_to_input_velocity.TIF path_to_output_velocity_at_term###.TIF

In [4]:
for BoxID in BOXIDS:
    #SET paths to the terminus box shapefiles and velocity data
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp"

    for vdate in ['2014_15', '2015_16', '2016_17']:
        vdir = vdate+'_velocity_dir_degree_yx.tif'
        vmag = vdate+'_velocity_mag.tif'
        
        #set input paths
        vdir_in = basepath+vdir
        vmag_in = basepath+vmag
    
        #SET output paths
        vdir_out = basepath+"Box"+BoxID+"/"+vdir
        vmag_out = basepath+"Box"+BoxID+"/"+vmag

        #SET velocity subset commands and print to check it
        v_subset_dir_cmd = 'gdalwarp -cutline '+terminusbox_path+' -crop_to_cutline '+vdir_in+" "+vdir_out
        v_subset_mag_cmd = 'gdalwarp -cutline '+terminusbox_path+' -crop_to_cutline '+vmag_in+" "+vmag_out
        print(v_subset_dir_cmd)
        print(v_subset_mag_cmd)

        #SUBSET velocity rasters
        subprocess.call(v_subset_dir_cmd, shell=True)
        subprocess.call(v_subset_mag_cmd, shell=True)
    
    print("Box"+BoxID)

gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box001/Box001.shp -crop_to_cutline /home/jukes/Documents/Sample_glaciers/2014_15_velocity_dir_degree_yx.tif /home/jukes/Documents/Sample_glaciers/Box001/2014_15_velocity_dir_degree_yx.tif
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box001/Box001.shp -crop_to_cutline /home/jukes/Documents/Sample_glaciers/2014_15_velocity_mag.tif /home/jukes/Documents/Sample_glaciers/Box001/2014_15_velocity_mag.tif
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box001/Box001.shp -crop_to_cutline /home/jukes/Documents/Sample_glaciers/2015_16_velocity_dir_degree_yx.tif /home/jukes/Documents/Sample_glaciers/Box001/2015_16_velocity_dir_degree_yx.tif
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box001/Box001.shp -crop_to_cutline /home/jukes/Documents/Sample_glaciers/2015_16_velocity_mag.tif /home/jukes/Documents/Sample_glaciers/Box001/2015_16_velocity_mag.tif
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box001/B

gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box235/Box235.shp -crop_to_cutline /home/jukes/Documents/Sample_glaciers/2015_16_velocity_dir_degree_yx.tif /home/jukes/Documents/Sample_glaciers/Box235/2015_16_velocity_dir_degree_yx.tif
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box235/Box235.shp -crop_to_cutline /home/jukes/Documents/Sample_glaciers/2015_16_velocity_mag.tif /home/jukes/Documents/Sample_glaciers/Box235/2015_16_velocity_mag.tif
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box235/Box235.shp -crop_to_cutline /home/jukes/Documents/Sample_glaciers/2016_17_velocity_dir_degree_yx.tif /home/jukes/Documents/Sample_glaciers/Box235/2016_17_velocity_dir_degree_yx.tif
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/Box235/Box235.shp -crop_to_cutline /home/jukes/Documents/Sample_glaciers/2016_17_velocity_mag.tif /home/jukes/Documents/Sample_glaciers/Box235/2016_17_velocity_mag.tif
Box235
gdalwarp -cutline /home/jukes/Documents/Sample_glaciers/B

Next, these subset velocity rasters are opened using the **rasterio** package and read into arrays. They are filtered for anomalous values and the velocity magnitudes are converted into weights. Then the **numpy.average()** function is used to calculated the weighted average flow directions where the flow directions of the pixels where the highest velocities are found are weighted more. 

The resulting average flow direction will be representative of the glacier's main flow. These directions will be used to rotate the images of the glaciers so that their flow is due right.

In [136]:
# #CREATE list of glacier average flow directions:
# boxes = []
# avg_rot = []
# max_mag = []
# years = []

# avg_rot_3 = []

# for BoxID in BOXIDS :
#     rot_angles = []
#     max_magnitudes = []
    
#     for vdate in ['2014_15', '2015_16', '2016_17']:
#         #READ velocity direction and magnitude data at terminus for each glacier into an array
#         vdir = vdate+'_velocity_dir_degree_yx.tif'
#         vmag = vdate+'_velocity_mag.tif'  
                
#         direction = rasterio.open(basepath+"Box"+BoxID+"/"+vdir, "r")
#         dir_array = direction.read()
#         #convert from radians to degree for 2016_17
#         if vdate == '2016_17':
#             dir_array = dir_array*180.0/np.pi

#         magnitude = rasterio.open(basepath+"Box"+BoxID+"/"+vmag, "r")
#         mag_array = magnitude.read()
       
#         #REMOVE anomalous values
#         #direction must be between 180 and -180 degrees
#         mask_dir = (dir_array < 180) & (dir_array > -180)
#         masked_dir = dir_array[mask_dir]
# #         print(dir_array.shape)
# #         print(masked_dir.shape)
# #         print(masked_dir.min(), masked_dir.max())

#         #magnitude must be between 0 and 10 m/d
#         mask_mag = (mag_array < 100) & (mag_array > 0)
#         masked_mag = mag_array[mask_mag]
# #         print(mag_array.shape)
# #         print(masked_mag.shape)
# #         print(masked_mag.min(), masked_mag.max())

#         #MAKE sure the two arrays are the same length
#         diff = np.diff([len(masked_dir), len(masked_mag)])[0]
# #         print(diff)   
#         if diff > 0:
#             masked_mag = masked_mag[:-diff]
# #             print(len(masked_mag))
# #             print(masked_dir)
#         if diff < 0:
#             masked_dir = masked_dir[:-diff]

#         #CALCULATE weights (0 - 1) from magnitudes
#         mag_range = masked_mag.max() - masked_mag.min()
#         stretch = 1/mag_range
#         weights = stretch*(masked_mag - masked_mag.min())
# #         print(weights.min(), weights.max()) #should be between 0 and 1
# #         print(weights.shape, masked_dir.shape)
        
#         #CALCULATE the weighted average rotation angle
#         avg_dir = np.average(masked_dir, weights=weights)

            
#         #APPEND the rotation angles to the dictionary
#         if masked_dir.min() == masked_dir.max():
#             rot_angles.append(masked_dir.min())
#         else:
#             rot_angles.append(avg_dir)
        
#         #APPEND the maximum flow magnitude
#         max_magnitudes.append(masked_mag.max())
        
#         #for all three years
#         years.append(vdate)
#         avg_rot_3.append(avg_dir)
#         boxes.append(BoxID)
#         max_mag.append(masked_mag.max())
#         print(vdate, avg_dir)
        
#     rot_angles = rot_angles[:-1]
#     max_magnitudes = max_magnitudes[:-1]
        
#     #APPEND the final values
# #     boxes.append(BoxID)
#     #grab maximum magnitude of all three years
# #     max_mag.append(np.max(max_magnitudes))
#     #grab the avg flow dir closest to the a box angle out of the three years
#     #load in box shape and grab coordinates using fiona
#     boxshp = fiona.open(basepath+'Box'+BoxID+'/Box'+BoxID+'.shp')
#     coordinates = boxshp[0]['geometry']['coordinates'][0]
#     coords = [xy[0:2] for xy in coordinates]
#     vertices = coords[:-1]
#     print(BoxID)
#     #grab vertex angles:
#     for i in range(0, len(vertices)):
#         #for the last vertex in the loop, use the original vertex as x2, y2
#         if i == 3:
#             x1, y1 = vertices[i]
#             x2, y2 = vertices[0]
#         else:
#             x1, y1 = vertices[i]
#             x2, y2 = vertices[i+1]
#         print(slope_angle(x1, y1, x2, y2))  
#     print('---------------')
# #create dataframe with the calculations
# velocities_df = pd.DataFrame(list(zip(boxes,years, avg_rot_3, max_mag)), 
#                              columns=['BoxID','Year','Flow_dir', 'Max_speed'])
# # velocities_df = pd.DataFrame(list(zip(boxes,avg_rot, max_mag)), columns=['BoxID','Flow_dir', 'Max_speed'])
# velocities_df = velocities_df.sort_values(by='BoxID')
# velocities_df = velocities_df.drop_duplicates()
# velocities_df

In [86]:
# #EXPORT MAX VELOCITY AND AVERAGE FLOW DIRECTION TO A .CSV FILE
# #write the data frame to csv file
# velocities_df.to_csv(path_or_buf = basepath+'Glacier_vel_3yrs_sample10.csv', sep=',', index=False)

In [31]:
# print(list(velocities_df.Flow_dir))

[24.437496185302734, 154.302001953125, 51.870487213134766, 119.75666809082031, -78.63849639892578, 18.988548278808594, -152.5367431640625, 114.55601501464844, -24.19788360595703, 21.693702697753906]


## 4) Rotate all images by flow direction

Read in the glacier velocity file as velocities_df if not already loaded:

In [139]:
velocities_df = pd.read_csv(basepath+'Glacier_velocities.csv', sep=',', dtype=str, usecols=[1,2,3])
velocities_df = velocities_df.set_index('BoxID')
velocities_df

,Flow_dir,Max_speed
BoxID,,
001,56.28428268432617,0.043774381279945374
002,155.9872283935547,3.583226442337036
004,-3.483433723449707,0.6230824589729309
033,142.1181640625,0.7716577649116516
120,-77.38639831542969,0.27788856625556946
174,12.677642822265625,0.9145031571388245
235,-145.85076904296875,0.15709678828716278
259,98.99927520751953,3.0749008655548096
277,-65.12018585205078,0.28601959347724915


In [36]:
#make rotated images directory in BoxID folders if it doesn't already exist
for BoxID in BOXIDS:
    if os.path.exists(downloadpath+"Box"+BoxID+'/rotated/'):
        print("Already exists.")
        #OTHERWISE, create the folder and download into it
    else:
        os.mkdir(downloadpath+"Box"+BoxID+'/rotated/')
        print("Folder made for Box"+BoxID)

Already exists.
Already exists.
Already exists.
Already exists.
Already exists.
Already exists.
Already exists.
Already exists.
Already exists.
Already exists.


In [44]:
#convert all files in reprojected folder to png from TIF
for BoxID in BOXIDS:
    command = 'cd '+downloadpath+'Box'+BoxID+'/reprojected/; '+'mogrify -format png *.TIF'
#     print(command)
    subprocess.call(command, shell=True)

In [52]:
#move box raster into reprojected folder:
for BoxID in BOXIDS:
    boxfile = 'Box'+BoxID+'_raster_cut.png'
    boxrasterpath = basepath+'Box'+BoxID+'/'+boxfile
    newpath = downloadpath+'Box'+BoxID+'/reprojected/'+boxfile
    
    shutil.copyfile(boxrasterpath, newpath)

In [140]:
#ROTATE THE IMAGES
for BoxID in BOXIDS:
    #for each file in the reprojected folder:
    for file in os.listdir(downloadpath+"Box"+BoxID+'/reprojected/'):
        if file.endswith('.png'):
            print(file)
            img  = Image.open(downloadpath+"Box"+BoxID+'/reprojected/'+file)
            #rotate the image by the flow direction from flowspeed_df
            rotated = img.rotate(-float(velocities_df.loc[BoxID, 'Flow_dir']))
            rotated.save(downloadpath+"Box"+BoxID+'/rotated/R_'+file)

LC80360042015104LGN00_B8_PS_Buffer001.png
LC80320052014153LGN00_B8_PS_Buffer001.png
LC80310052014242LGN00_B8_PS_Buffer001.png
LC80350052013267LGN00_B8_PS_Buffer001.png
LC80350052016180LGN00_B8_PS_Buffer001.png
LC80350052014126LGN00_B8_PS_Buffer001.png
LC80310052016280LGN00_B8_PS_Buffer001.png
LC80320052014249LGN00_B8_PS_Buffer001.png
LC80350052015145LGN00_B8_PS_Buffer001.png
LC80360042015264LGN00_B8_PS_Buffer001.png
LC80350052015225LGN00_B8_PS_Buffer001.png
LC80330052014176LGN00_B8_PS_Buffer001.png
LC80320052015092LGN00_B8_PS_Buffer001.png
LC80320052015284LGN00_B8_PS_Buffer001.png
LC80350052013123LGN01_B8_PS_Buffer001.png
LC80360042015168LGN00_B8_PS_Buffer001.png
Box001_raster_cut.png
LC80330052016070LGN00_B8_PS_Buffer001.png
LC80310052015229LGN00_B8_PS_Buffer001.png
LC80330052014160LGN00_B8_PS_Buffer001.png
LC80330052017072LGN00_B8_PS_Buffer001.png
LC80310052015277LGN00_B8_PS_Buffer001.png
LC80350052015177LGN00_B8_PS_Buffer001.png
LC80320052013246LGN00_B8_PS_Buffer001.png
LC8032005201

LC80350052015225LGN00_B8_PS_Buffer002.png
LC80330052014192LGN00_B8_PS_Buffer002.png
LC80330052014272LGN00_B8_PS_Buffer002.png
LC80330052014176LGN00_B8_PS_Buffer002.png
LC80350052016276LGN00_B8_PS_Buffer002.png
LC80340052013260LGN00_B8_PS_Buffer002.png
LC80320052014249LGN00_B8_PS_Buffer002.png
LC80330052015211LGN00_B8_PS_Buffer002.png
LC80330052016262LGN00_B8_PS_Buffer002.png
LC80340052015202LGN00_B8_PS_Buffer002.png
LC80320052015236LGN00_B8_PS_Buffer002.png
LC80300052014251LGN00_B8_PS_Buffer002.png
LC80350052014078LGN00_B8_PS_Buffer002.png
LC80310052014082LGN00_B8_PS_Buffer002.png
LC80310052016280LGN00_B8_PS_Buffer002.png
LC80310052015245LGN00_B8_PS_Buffer002.png
LC80350052016260LGN00_B8_PS_Buffer002.png
LC80350052016212LGN00_B8_PS_Buffer002.png
LC80350052017086LGN00_B8_PS_Buffer002.png
LC80300052016081LGN00_B8_PS_Buffer002.png
LC80340052016237LGN00_B8_PS_Buffer002.png
LC80350052016084LGN00_B8_PS_Buffer002.png
LC80340052013148LGN00_B8_PS_Buffer002.png
LC80310052015181LGN00_B8_PS_Buffer

LC80340052016237LGN00_B8_PS_Buffer004.png
LC80330052016150LGN00_B8_PS_Buffer004.png
LC80310052014178LGN00_B8_PS_Buffer004.png
LC80290062015247LGN00_B8_PS_Buffer004.png
LC80290062014068LGN00_B8_PS_Buffer004.png
LC80320052014185LGN00_B8_PS_Buffer004.png
LC80320052014249LGN00_B8_PS_Buffer004.png
LC80310052015165LGN00_B8_PS_Buffer004.png
LC80340052014167LGN00_B8_PS_Buffer004.png
LC80070142015045LGN00_B8_PS_Buffer033.png
LC80080142014161LGN00_B8_PS_Buffer033.png
LC80080142015212LGN00_B8_PS_Buffer033.png
LC80080142014273LGN00_B8_PS_Buffer033.png
LC80090142016206LGN00_B8_PS_Buffer033.png
LC80090132014184LGN00_B8_PS_Buffer033.png
LC80080142015196LGN00_B8_PS_Buffer033.png
LC80080142016231LGN00_B8_PS_Buffer033.png
LC80090132014216LGN00_B8_PS_Buffer033.png
LC80080142016183LGN00_B8_PS_Buffer033.png
LC80070142014122LGN00_B8_PS_Buffer033.png
LC80070142015141LGN00_B8_PS_Buffer033.png
LC80070142015269LGN00_B8_PS_Buffer033.png
LC80090142014312LGN00_B8_PS_Buffer033.png
LC80080142017057LGN00_B8_PS_Buffer

LC82330172013150LGN00_B8_PS_Buffer120.png
LC82320172014098LGN00_B8_PS_Buffer120.png
LC82320172016312LGN00_B8_PS_Buffer120.png
LC82320172015341LGN00_B8_PS_Buffer120.png
LC82320172015021LGN00_B8_PS_Buffer120.png
LC82320182013351LGN00_B8_PS_Buffer120.png
LC82320172016104LGN00_B8_PS_Buffer120.png
LC82320172014178LGN00_B8_PS_Buffer120.png
LC82330172015300LGN00_B8_PS_Buffer120.png
Box120_raster_cut.png
LC82330172016175LGN00_B8_PS_Buffer120.png
LC82320182015341LGN00_B8_PS_Buffer120.png
LC82330172013230LGN00_B8_PS_Buffer120.png
LC82330172014009LGN00_B8_PS_Buffer120.png
LC82320172016328LGN00_B8_PS_Buffer120.png
LC82330172015140LGN00_B8_PS_Buffer120.png
LC82320182017026LGN00_B8_PS_Buffer120.png
LC82330172017017LGN00_B8_PS_Buffer120.png
LC82320182017074LGN00_B8_PS_Buffer120.png
LC82330172014297LGN00_B8_PS_Buffer120.png
LC82330172016127LGN00_B8_PS_Buffer120.png
LC82330172016255LGN00_B8_PS_Buffer120.png
LC82330172015252LGN00_B8_PS_Buffer120.png
LC82330172014249LGN00_B8_PS_Buffer120.png
LC8232018201

LC82320152016200LGN00_B8_PS_Buffer235.png
LC82330152015140LGN00_B8_PS_Buffer235.png
LC82330152015220LGN00_B8_PS_Buffer235.png
LC82330152016127LGN00_B8_PS_Buffer235.png
LC80010152016182LGN00_B8_PS_Buffer259.png
LC80010152016278LGN00_B8_PS_Buffer259.png
LC82320152016200LGN00_B8_PS_Buffer259.png
LC80010152016198LGN00_B8_PS_Buffer259.png
LC80010152015291LGN00_B8_PS_Buffer259.png
LC80010152013317LGN00_B8_PS_Buffer259.png
LC82320152015197LGN00_B8_PS_Buffer259.png
LC82330152016127LGN00_B8_PS_Buffer259.png
LC80010152017040LGN00_B8_PS_Buffer259.png
LC82330152015188LGN00_B8_PS_Buffer259.png
LC82320152017026LGN00_B8_PS_Buffer259.png
LC82320152015245LGN00_B8_PS_Buffer259.png
LC80010152016150LGN00_B8_PS_Buffer259.png
LC80010152015051LGN00_B8_PS_Buffer259.png
LC82330152015028LGN00_B8_PS_Buffer259.png
LC82330152013246LGN00_B8_PS_Buffer259.png
LC80010152015211LGN00_B8_PS_Buffer259.png
LC80010152016294LGN00_B8_PS_Buffer259.png
Box259_raster_cut.png
LC80010152015227LGN00_B8_PS_Buffer259.png
LC8001015201

LC80130022016170LGN00_B8_PS_Buffer531.png
LC80120022016131LGN00_B8_PS_Buffer531.png
LC80070032014106LGN00_B8_PS_Buffer531.png
LC80060032016105LGN00_B8_PS_Buffer531.png
LC80070032017082LGN00_B8_PS_Buffer531.png
LC80120022013122LGN01_B8_PS_Buffer531.png
LC80100022014095LGN00_B8_PS_Buffer531.png
LC80160012015124LGN00_B8_PS_Buffer531.png
LC80130022014196LGN00_B8_PS_Buffer531.png
LC80100022014191LGN00_B8_PS_Buffer531.png
LC80100022017087LGN00_B8_PS_Buffer531.png
LC80070032016096LGN00_B8_PS_Buffer531.png
LC80110022016268LGN00_B8_PS_Buffer531.png
LC80130022016138LGN00_B8_PS_Buffer531.png
LC80120022017101LGN00_B8_PS_Buffer531.png
LC80070022016160LGN00_B8_PS_Buffer531.png
LC80130022017108LGN00_B8_PS_Buffer531.png
LC80140012014251LGN00_B8_PS_Buffer531.png
LC80160012014233LGN00_B8_PS_Buffer531.png
LC80070032015221LGN00_B8_PS_Buffer531.png
LC80090022016238LGN00_B8_PS_Buffer531.png
LC80100022017119LGN00_B8_PS_Buffer531.png
LC80130022013113LGN01_B8_PS_Buffer531.png
LC80100022013124LGN01_B8_PS_Buffer

LC80160012015204LGN00_B8_PS_Buffer531.png
LC80080022016135LGN00_B8_PS_Buffer531.png
LC80160012014105LGN00_B8_PS_Buffer531.png
LC80100022014223LGN00_B8_PS_Buffer531.png
LC80090022017112LGN00_B8_PS_Buffer531.png
LC80080022015116LGN00_B8_PS_Buffer531.png


## 5) Resize images to minimum dimensions

In [144]:
# #Make sure resized folders are removed
# for BoxID in BOXIDS:
#     if os.path.exists(downloadpath+"Box"+BoxID+'/resized/'):
#         shutil.rmtree(downloadpath+"Box"+BoxID+'/resized/', ignore_errors=False, onerror=None)
#     else:
#         print("Resized folder already removed")

In [141]:
for BoxID in BOXIDS:
    dimensions_x = []
    dimensions_y = []
    images = os.listdir(downloadpath+"Box"+BoxID+'/rotated/')
    for image in images:
        img = mpimg.imread(downloadpath+"Box"+BoxID+'/rotated/'+image)
        dimensions_x.append(img.shape[1])
        dimensions_y.append(img.shape[0])
    
    #find minimum dimensions
    min_y = np.min(dimensions_y)
    min_x = np.min(dimensions_x)
    index_y = dimensions_y.index(min_y)
    index_x = dimensions_x.index(min_x)
          
    if index_x != index_y:
        print('Something is funky with the image dimesions for Box'+BoxID)
    else:
        crop_y = dimensions_y[index_y]
        crop_x = dimensions_x[index_y]

        #crop each image if the dimensions are larger than the minimum
        for image in images:
            img = mpimg.imread(downloadpath+"Box"+BoxID+'/rotated/'+image)
            if img.shape[1] > crop_x or img.shape[0] > crop_y:
                #calculate difference, and divide by 2 to get amount of rows to remove by
                diffx_half = (img.shape[1] - crop_x)/2
                diffy_half = (img.shape[0] - crop_y)/2
#                 print(diffx_half, diffy_half)
                
                #if the difference is a half pixel, make sure to remove the full value from the first side only
                if int(diffx_half) != diffx_half:
                    #remember for image slicing y is the first dimension, x is the second
                    img_cropx = img[:, int(diffx_half):-int(diffx_half)-1]
                #otherwise remove it from both sides:
                else:
                    img_cropx = img[:, int(diffx_half):-int(diffx_half)]
                
                #same for y
                if int(diffy_half) != diffy_half:   
                    img_cropy = img_cropx[int(diffy_half):-int(diffy_half)-1, :]
                #otherwise remove it from both sides:
                else:
                    img_cropy = img_cropx[int(diffy_half):-int(diffy_half), :]
                
#                 print(BoxID, crop_y, crop_x)
#                 print(BoxID, img_cropy.shape)
                    
                #save over original images
                scipy.misc.imsave(downloadpath+"Box"+BoxID+'/rotated/'+image, img_cropy)

/home/jukes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [159]:
#convert all final files to pgm
for BoxID in BOXIDS:
    command = 'cd '+downloadpath+'Box'+BoxID+'/rotated/; '+'mogrify -format pgm *.png'
#     print(command)
    subprocess.call(command, shell=True)

## Now we're ready for 2D WTMM analysis!

## 6) Run Tcl scripts: 

Pull in the input BoxIDs from above.

In [148]:
inputIDs = " ".join(BOXIDS)
print(inputIDs)

001 002 004 033 120 174 235 259 277 531


### Run scr_gaussian.tcl with BoxIDs as input

In [164]:
scr_gaussian = '/home/akhalil/src/xsmurf-2.7/main/xsmurf -nodisplay /home/jukes/Documents/Scripts/scr_gaussian.tcl '+inputIDs
print(scr_gaussian)
subprocess.call(scr_gaussian, shell=True)

/home/akhalil/src/xsmurf-2.7/main/xsmurf -nodisplay /home/jukes/Documents/Scripts/scr_gaussian.tcl 001 002 004 033 120 174 235 259 277 531


0

### Run terminus_pick.tcl with thresholds and BoxIDs as input

In [174]:
size_thresh = 0.8
mod_thresh = 0.8
terminus_pick = '/home/akhalil/src/xsmurf-2.7/main/xsmurf -nodisplay /home/jukes/Documents/Scripts/terminus_pick.tcl '+str(size_thresh)+' '+str(mod_thresh)+' '+inputIDs
# print(terminus_pick)
subprocess.call(terminus_pick, shell=True)

0